# A Multi Target Random Forest Regression (MTRT)

In this section we develop a multi target random forest regression for the prediction of a minimum and maximum price of laptop models.


## 1. Import Statements


In [1]:
import pandas as pd
import numpy as np
import math
import io

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pydotplus

from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor 

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import plot_tree
from sklearn.externals.six import StringIO

from IPython.display import SVG
from graphviz import Source
from IPython.display import display

## Edit Print options
desired_width = 500
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 600)

//anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


## 2. Dealing with missing values



In [2]:
## Importing the preprocessed data
missing_values = ["n/a", "na", "--","NAN"," ","nan","NaN",""]
data = pd.read_csv('datafile2.csv',na_values = missing_values)
data2 = pd.read_csv('datafile2_test.csv',na_values = missing_values)

## Removing missing values
# data.dropna()

## Just a temporary dataset to test the capabilities of the model
temp_data = data[['id','brand','base_name','os_details','gpu_brand','cpu_brand','cpu_type_name','gpu_type','screen_surface','screen_size','pixels_x',
                  'touchscreen','discrete_gpu','gpu','ram','ssd','detachable_keyboard','weight','storage',
                  'cpu_benchmark','gpu_benchmark','cpu_GHZ','cpu_core','threading','pc_name','min_price','max_price']]

test_data = data2[['id','brand','base_name','os_details','gpu_brand','cpu_brand','cpu_type_name','gpu_type','screen_surface','screen_size','pixels_x',
                   'touchscreen','discrete_gpu','gpu','ram','ssd', 'detachable_keyboard','weight','storage',
                   'cpu_benchmark','gpu_benchmark','cpu_GHZ','cpu_core','threading','pc_name']]

concat_df = pd.concat([temp_data , test_data])
concat_df.to_csv("/Users/Simon/Documents/GitHub/adana123/concat_df1.csv",header=True,index=True)

##### TRAINING DATA #####
## Filling in missing values: CATEGORICAL & BINARY VARIABLES
detach_key = int(temp_data.loc[:,'detachable_keyboard'].mode())
temp_data.loc[:,'detachable_keyboard'] = temp_data.loc[:,'detachable_keyboard'].fillna(0)

temp_data.loc[:,'gpu_brand'] = temp_data.loc[:,'gpu_brand'].fillna("Unknown", inplace = False) 
temp_data['cpu_type_name'] = temp_data.apply(lambda row: row['cpu_brand'] if pd.isnull(row['cpu_type_name']) 
                                             else row['cpu_type_name'], axis=1)
temp_data['gpu_type'] = temp_data['gpu_type'].fillna("No GPU", inplace = False) 
temp_data.loc[:,'screen_surface'] = temp_data.loc[:,'screen_surface'].fillna("Unknown", inplace = False) 
temp_data['os_details'] = temp_data['os_details'].fillna("Unknown", inplace = False) 
temp_data['cpu_core'] = temp_data['cpu_core'].fillna(temp_data['cpu_core'].mode())

# ## Filling in missing values: NUMERICAL VARIABLES
temp_data['weight'] = temp_data['weight'].fillna(temp_data['weight'].mean())
temp_data['cpu_benchmark'] = temp_data['cpu_benchmark'].fillna(temp_data['cpu_benchmark'].mean()) 
temp_data['gpu_benchmark'] = temp_data['gpu_benchmark'].fillna(temp_data['gpu_benchmark'].mean())
temp_data['cpu_GHZ'] = temp_data['cpu_GHZ'].fillna(temp_data['cpu_GHZ'].mean())
pixel_mode = int(temp_data.loc[:,'pixels_x'].mode())
temp_data['pixels_x'] = temp_data['pixels_x'].fillna(pixel_mode) #Modus! Most frequent number! (median)

## Showing the data
# data.head()

########################################################################################################

##### TRAINING & TESTING DATA #####
## Filling in missing values: CATEGORICAL & BINARY VARIABLES
detach_key = int(temp_data.loc[:,'detachable_keyboard'].mode())
concat_df.loc[:,'detachable_keyboard'] = concat_df.loc[:,'detachable_keyboard'].fillna(detach_key)
# print(concat_df['detachable_keyboard'])
concat_df['gpu_brand'] = concat_df['gpu_brand'].fillna("Unknown", inplace = False) 
concat_df['cpu_type_name'] = concat_df.apply(lambda row: row['cpu_brand'] if pd.isnull(row['cpu_type_name']) 
                                             else row['cpu_type_name'], axis=1)

concat_df['gpu_type'] = concat_df['gpu_type'].fillna("No GPU", inplace = False)
concat_df['gpu'] = concat_df['gpu'].fillna("No GPU", inplace = False) 
concat_df['screen_surface'] = concat_df['screen_surface'].fillna("Unknown", inplace = False) 
concat_df['os_details'] = concat_df['os_details'].fillna("Unknown", inplace = False) 
concat_df['cpu_core'] = concat_df['cpu_core'].fillna(temp_data['cpu_core'].mode())

## Filling in missing values: NUMERICAL VARIABLES
concat_df['weight'] = concat_df['weight'].fillna(temp_data['weight'].mean())
concat_df['cpu_benchmark'] = concat_df['cpu_benchmark'].fillna(temp_data['cpu_benchmark'].mean()) 
concat_df['gpu_benchmark'] = concat_df['gpu_benchmark'].fillna(temp_data['gpu_benchmark'].mean())
concat_df['cpu_GHZ'] = concat_df['cpu_GHZ'].fillna(temp_data['cpu_GHZ'].mean())

pixel_mode = int(temp_data.loc[:,'pixels_x'].mode())
concat_df.loc[:,'pixels_x'] = concat_df.loc[:,'pixels_x'].fillna(pixel_mode)



temp_data.to_csv("/Users/Simon/Documents/GitHub/adana123/temp_data.csv",header=True,index=True)
concat_df.to_csv("/Users/Simon/Documents/GitHub/adana123/concat_df2.csv",header=True,index=True)


#### Mean only from training set!

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

## 3. Feature Importance (Selection)

## 4. Splitting Data & Fitting Data

In [3]:
#### Training Data ####
## Seperating categorical - binary - numerical variables
X_cat = temp_data[['cpu_core','brand','cpu_brand','gpu_brand','gpu','cpu_type_name','screen_surface']]#,'gpu_type','os_details','base_name'
X_bin = temp_data[['touchscreen','discrete_gpu','detachable_keyboard','threading']]
X_num = temp_data[['id','screen_size','ram','ssd','storage','weight','cpu_GHZ','pixels_x','cpu_benchmark','gpu_benchmark']]

## Creating dummy variables:
X_cat_dummies = pd.get_dummies(X_cat, drop_first=True)

## Merging input data
Xa = pd.concat([X_cat_dummies,X_bin,X_num], axis=1)
Xb = pd.concat([X_cat,X_bin,X_num], axis=1)
# X = pd.concat([X_cat_dummies,X_num], axis=1)
# X = X_cat_dummies

## Defining output data
Ya = temp_data[['min_price','max_price']]
Yb = Ya
# Yb = temp_data[['min_price','diff_price']]

########################################################################################
## Seperating categorical - binary - numerical variables
X_cat_test = concat_df[['cpu_core','brand','cpu_brand','gpu_brand','gpu','cpu_type_name','screen_surface']]#'pc_name','gpu_type','os_details','base_name'
X_bin_test = concat_df[['touchscreen','discrete_gpu','detachable_keyboard','threading']]
X_num_test = concat_df[['screen_size','ram','ssd','storage','weight','cpu_GHZ','pixels_x','cpu_benchmark','gpu_benchmark']]

## Creating dummy variables:
X_cat_dummies_test = pd.get_dummies(X_cat_test, drop_first=True)

## Merging input data
X_full = pd.concat([X_cat_dummies_test,X_bin_test,X_num_test], axis=1)
X_train = X_full.head(len(data))
X_test = X_full.tail(len(data2))

########################################################################################
## Splitting the dataset into train and test sets
# Predict Minimum and Maximum prices
X_train_a, X_test_a, Y_train_a, Y_test_a = train_test_split(Xa, Ya, test_size=0.3, random_state=42)


#Split without creating dummies for easy analysis
X_train_b, X_test_b, Y_train_b, Y_test_b = train_test_split(Xb, Yb, test_size=0.3, random_state=42)

trainingdata = pd.concat([X_train_b, Y_train_b], axis=1, sort=False)
testingdata = pd.concat([X_test_b, Y_test_b], axis=1, sort=False)

trainingdata.to_csv("/Users/Simon/Documents/GitHub/adana123/trainingdata.csv",header=True,index=True)
testingdata.to_csv("/Users/Simon/Documents/GitHub/adana123/testingdata.csv",header=True,index=True)


## 2 models
del X_train_a['id'] #need to delete a useless feature first
del X_test_a['id']

Model1 = RandomForestRegressor(n_estimators = 400,criterion='mae',random_state = 42)
Model1.fit(X_train_a,Y_train_a)

# Model2 = RandomForestRegressor(n_estimators = 200,criterion='mae',random_state = 0)
# Model2.fit(X_train_b,Y_train_b)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

## 5. Measuring Mean Summed Absolute Error

### A: MSAE for predicting minimum and maximum  price

In [4]:
## Predicting the target values
Y_pred = Model1.predict(X_test_a)

## Calculating the score
MAE = mean_absolute_error(Y_test_a, Y_pred, multioutput='raw_values')
print(MAE[0]+MAE[1])

## Turn DF to array
Y_test_array = Y_test_a.to_numpy()

Sum_error = 0
Test_Results = []

for i in range(len(Y_pred)):
    
    temp_test_results = []
    
#     Y_id = X_test_b['id'][i]
    Y_pred_min = Y_pred[i][0]
    Y_pred_max = Y_pred[i][1]
    Y_test_min = Y_test_array[i][0]
    Y_test_max = Y_test_array[i][1]
    
    temp_test_results = [Y_pred_min,Y_test_min,Y_pred_max,Y_test_max]
    Test_Results.append(temp_test_results)
    
    Error_min = abs(Y_pred_min - Y_test_min)
    Error_max = abs(Y_pred_max - Y_test_max)
    Total_error = Error_min + Error_max
    
    Sum_error = Sum_error + Total_error
    counter = i

Avg_error = Sum_error / counter
MSAE = Avg_error
print(MSAE)



334.6748986111111
336.87670715460547


In [5]:
# Array --> Datafame
Test_Results = pd.DataFrame(Test_Results,columns=['pred_min_price','act_min_price','pred_max_price','act_max_price'])

# Reset Indexes of DF
X_test_b.reset_index(inplace=True) 
Test_Results.reset_index(inplace=True,drop=True) 

# Copy the ID column
Test_Results.loc[:,'id'] = X_test_b.loc[:,'id']

# Set ID as new Index
X_test_b.set_index('id')
Test_Results.set_index('id')

# Merge the features with the results
testingdata_results = X_test_b.merge(Test_Results, left_index=True, right_index=True)
# testingdata_results = pd.concat([X_test_b, Test_Results], axis=1, sort=False)
# testingdata_results.drop(['id_x'],axis=1)
# testingdata_results.drop(['id_y'],axis=1)
testingdata_results.to_csv("/Users/Simon/Documents/GitHub/adana123/testingdata_results.csv",header=True,index=True)
testingdata_results.head(len(testingdata_results))

,index,cpu_core,brand,cpu_brand,gpu_brand,gpu,cpu_type_name,screen_surface,touchscreen,discrete_gpu,detachable_keyboard,threading,id_x,screen_size,ram,ssd,storage,weight,cpu_GHZ,pixels_x,cpu_benchmark,gpu_benchmark,pred_min_price,act_min_price,pred_max_price,act_max_price,id_y
0,480,QUAD-CORE,DELL,AMD,AMD,AMD RADEON VEGA 10,RYZEN 7,GLOSSY,1,0,0.0,1.0,23196,13.3,12,256,256,3.860000,2.200000,1920,6903.000000,1571.000000,608.654925,512.99,617.229150,517.99,23196
1,449,HEXA-CORE,MSI,INTEL,NVIDIA,NVIDIA GEFORCE GTX 1060,CORE I7,MATTE,0,1,0.0,1.0,26773,15.6,16,512,512,4.140000,2.200000,1920,10418.000000,10210.000000,1664.221262,2099.00,1756.052225,2099.00,26773
2,475,QUAD-CORE,HP,INTEL,INTEL,INTEL UHD 620,CORE I5,GLOSSY,1,0,0.0,1.0,27887,15.6,12,0,1000,4.230000,1.600000,1366,6372.000000,888.000000,581.207662,621.00,591.804862,621.99,27887
3,434,DUAL-CORE,APPLE,INTEL,INTEL,INTEL HD 6000,CORE I5,GLOSSY,0,0,0.0,1.0,28480,11.6,4,128,128,2.380000,1.600000,1366,2142.000000,829.000000,387.233413,418.00,416.794688,418.00,28480
4,368,QUAD-CORE,ACER,INTEL,INTEL,INTEL HD,ATOM,MATTE,0,0,0.0,0.0,29847,15.6,4,16,16,4.300000,1.040000,1366,1196.000000,10.000000,220.768800,197.97,227.550888,197.99,29847
5,55,DUAL-CORE,ACER,INTEL,NVIDIA,NVIDIA GEFORCE 940M,CORE I5,MATTE,0,1,0.0,1.0,7495,15.6,8,0,1000,5.290000,2.200000,1920,2546.000000,1048.000000,720.513325,789.00,741.496975,789.00,7495
6,322,QUAD-CORE,ACER,INTEL,NVIDIA,NVIDIA GEFORCE GTX 1050 TI,CORE I5,MATTE,0,1,0.0,0.0,23606,15.6,8,256,256,5.950000,2.500000,1920,5260.000000,6657.000000,786.062275,705.98,800.921738,748.00,23606
7,76,HEXA-CORE,ALIENWARE,INTEL,NVIDIA,NVIDIA GEFORCE GTX 1060,CORE I7,MATTE,0,1,0.0,1.0,26135,17.3,8,256,1256,9.740000,2.200000,1920,10418.000000,10210.000000,1088.020400,1699.00,1113.178362,1699.00,26135
8,504,DUAL-CORE,LENOVO,INTEL,INTEL,INTEL HD 500,CELERON,MATTE,0,0,0.0,0.0,20733,15.6,4,0,1000,4.850000,1.100000,1366,1123.000000,317.000000,310.395350,299.99,320.116725,342.00,20733
9,347,QUAD-CORE,ASUS,INTEL,NVIDIA,NVIDIA GEFORCE GTX 1050,CORE I7,MATTE,0,1,0.0,1.0,17467,15.6,16,256,1256,5.100000,2.800000,1920,7069.000000,5595.000000,1269.918425,938.00,1347.512575,938.00,17467


### B: MSAE for predicting minimum price and price difference

**Result:** No significant improvement was obtained. This is as expected because the regression tree considers the two outputs, min and max price, at the same time. They are not considered independent and the splits are based on improving both outputs.

In [6]:
# Y_pred = Model2.predict(X_test_b)

# ## Turn DF to array
# Y_test_array = Y_test_b.to_numpy()

# Sum_error = 0
# Y_pred_min_array = []
# for i in range(len(Y_pred)):
# #     print(Y_pred[i],Y_test_array[i])
#     Y_pred_min = Y_pred[i][0]
#     Y_pred_max = Y_pred_min + Y_pred[i][1]
#     Y_test_min = Y_test_array[i][0]
#     Y_test_max = Y_test_min + Y_test_array[i][1]
    
#     Y_pred_min_array.append(Y_pred_min)
    
#     Error_min = abs(Y_pred_min - Y_test_min)
#     Error_max = abs(Y_pred_max - Y_test_max)
#     Total_error = Error_min + Error_max
    
#     Sum_error = Sum_error + Total_error
#     counter = i

# Avg_error = Sum_error / counter
# MSAE = Avg_error
# print('The MSAE = ', round(MSAE,3))



## 6. Visualization of Model and Data

In [7]:
## Printing the decision tree
# FEATURE_NAMES = X_train.columns[:]


# dot_data = export_graphviz(Model1,
#                                out_file=None,
#                                feature_names=FEATURE_NAMES,
#                                filled = True)
# graph = Source(dot_data)

# # dot_data = io.StringIO()
# # export_graphviz(Model1, out_file=dot_data, rounded=True, filled=True)
# # filename = "tree.png"
# # pydotplus.graph_from_dot_data(dot_data.getvalue()).write_png(filename)
# # plt.figure(figsize=(300,100))
# # img = mpimg.imread(filename)
# # imgplot = plt.imshow(img)
# # plt.show()

# plt.figure(figsize=(120,50))
# plot_tree(Model1, feature_names=FEATURE_NAMES,fontsize=11, rounded=True)
# plt.show()

In [8]:
# Visualising the Random Forest Regression results 
  
# arange for creating a range of values 
# from min value of x to max  
# value of x with a difference of 0.01  
# between two consecutive values 
# X_grid = np.arange(X_test, max(x), 0.01)  
# X_ram
  
# reshape for reshaping the data into a len(X_grid)*1 array,  
# i.e. to make a column out of the X_grid value                   
# X_grid = X_grid.reshape((len(X_grid), 1)) 
  
# Scatter plot for original data 
# plt.scatter(x, y, color = 'blue')   
  
# plot predicted data 
# plt.plot(Y_pred_min_array,  
#          color = 'green')  
# plt.title('Random Forest Regression') 
# plt.xlabel('Position level') 
# plt.ylabel('Max_price') 
# plt.show()

## 7. Predicting the test set

In [9]:
Model_Final = RandomForestRegressor(n_estimators = 1500,criterion='mae',random_state = 1)
Model_Final.fit(X_train,Ya)
Y_pred_test = Model_Final.predict(X_test)

RESULTS = []
for i in range(len(data2)):
    temp_result = []
    temp_result.append(data2['id'][i])
    temp_result.append(Y_pred_test[i][0])
    temp_result.append(Y_pred_test[i][1])
    RESULTS.append(temp_result)

RESULTS = pd.DataFrame(RESULTS)
print(RESULTS)



         0            1            2
0    28807  1134.870157  1201.351163
1    22559   387.260830   404.441187
2    28647   812.627720   837.732520
3    22141   539.488377   556.233570
4    26116  2120.133467  2277.885373
5    27111  1426.302800  1474.115647
6    23420   623.497333   649.130347
7    21464   176.708933   182.493140
8    29405   724.204870   760.924067
9    27107   407.119153   418.358347
10   26141   967.903400   999.354357
11   25928  1693.143120  1780.425607
12   24845   712.290403   772.846153
13   28804   359.293627   370.870473
14   26772   610.062610   616.740990
15   27413   921.349217   958.715263
16    7261   879.480957   924.598847
17   31424   799.589710   823.023227
18    3940   407.876900   418.563843
19   16238   533.073810   550.847843
20   29407  1291.823747  1339.334510
21   24011   387.670517   405.910307
22    8542   390.590153   397.930147
23   14448   720.620330   735.171427
24   19611   243.432913   252.807533
25   30068  1197.941173  1259.165620
2

In [10]:
RESULTS.to_csv("/Users/Simon/Documents/GitHub/adana123/RESULTS.csv",header=False,index=False)